In [1]:
import pandas as pd
import pulp as plp

In [2]:
df = pd.read_csv(r'C:\Users\jpcam\Área de Trabalho\projeto linear programming\matérias para o algoritmo.csv', delimiter = ';')

In [3]:
# DEFININDO PROBLEMA

prob = plp.LpProblem('maximizar_créditos', plp.LpMaximize)

In [4]:
# DEFININDO VARIÁVEIS DE DECISÃO

variaveis = [plp.LpVariable('x{}'.format(str(i)), 0, 1, cat='Integer') for i in range(df.shape[0])]

creditos = []
for i in range(df.shape[0]):
    creditos += [df.iloc[i][1]]

In [5]:
# ESCREVENDO FUNÇÃO-OBJETIVO

prob += plp.lpSum(variaveis[i]*creditos[i] for i in range(df.shape[0]))

In [6]:
# CRIANDO AS PRIEMEIRAS RESTRIÇÕES

# restrições de horários iguais em matérias diferentes

for k in [2,4,6]:
    for i in range(df.shape[0]):
        h = str(df.iloc[i][k])+str(df.iloc[i][k+1])
        for j in range(df.shape[0]):
            if j > i:
                h1 = str(df.iloc[j][2])+str(df.iloc[j][3])
                h2 = str(df.iloc[j][4])+str(df.iloc[j][5])
                h3 = str(df.iloc[j][6])+str(df.iloc[j][7])
                if h != '--':
                    if h == h1 or h == h2 or h == h3:
                        prob += variaveis[i] + variaveis[j] <= 1
                        
# restrições de matérias iguais em horários diferentes

for i in range(df.shape[0]):
    for j in range(df.shape[0]):
        if i != j and df.iloc[i][0] == df.iloc[j][0]:
                prob += variaveis[i] + variaveis[j] <= 1

In [7]:
# RODANDO MODELO E ATUALIZANDO AS RESTRIÇÕES PARA GERAR MAIS GRADES

contagem = 0
prob.solve()

while contagem < 5:
    
    prob.solve()
    
    # CRIANDO NOVA GRADE VAZIA
    
    dias = ['horário','SEG', 'TER', 'QUA', 'QUI', 'SEX', 'SAB']
    horarios = {'horário': ['8:00','8:55','10:00','10:55','12:00','12:55','14:00','14:55','16:00','16:55','18:00','18:55','19:00','19:50','20:50','21:40']}

    grade = pd.DataFrame(horarios,columns=dias)
    
    # PREENCHENDO NOVA GRADE

    for i in prob.variables():
        if i.varValue == 1:
            j = int(i.name.replace("x",""))
            for k in [2,4,6]:
                if df.iloc[j][k] != '-':
                    dia = df.iloc[j][k]
                    horario = df.iloc[j][k+1]
                    if horario[0] == 'M':
                        for n in horario[1:]:
                            grade.iloc[int(n)][int(dia)-1] = df.iloc[j][0]
                    if horario[0] == 'T':
                        for n in horario[1:]:
                            grade.iloc[int(n)+4][int(dia)-1] = df.iloc[j][0]
                    if horario[0] == 'N':
                        for n in horario[1:]:
                            grade.iloc[int(n)+11][int(dia)-1] = df.iloc[j][0]

    grade.to_csv(r'C:\Users\jpcam\Área de Trabalho\grade ' + str(contagem+1) + '.csv',sep=';',encoding='latin1')

    contagem = contagem + 1

    # ESCREVER NOVAS RESTRIÇÕES AQUI
    
    nova_restriçao = []

    for i in prob.variables():
        if i.varValue == 1:
            nova_restriçao = nova_restriçao + [i]
    
    prob += plp.lpSum(nova_restriçao[i] for i in range(len(nova_restriçao))) <= len(nova_restriçao)-1
    